In [64]:
import sys
import requests
import pandas as pd 

sell_SL_diff_7d = []
sell_SL_diff_1d = []
buy_SL_diff_7d = []
buy_SL_diff_1d = []
sell_sold_5pct_1d = []
profit_est = []

url_items_tp_data = (   'https://api.datawars2.ie/gw2/v1/items/json?fields='
                        'id,name,buy_price,sell_price,roi,profit,'
                        '7d_buy_listed,7d_buy_sold,7d_sell_listed,7d_sell_sold,'
                        '1d_buy_listed,1d_buy_sold,1d_sell_listed,1d_sell_sold')


result = requests.get(url_items_tp_data)

if(result.status_code == 200):
    data = result.json()
    df = pd.json_normalize(data)

else:
	sys.exit("Status code not 200")



for i, row in df.iterrows():

    sell_SL_diff_7d.append(row['7d_sell_sold'] - row['7d_sell_listed'])
    sell_SL_diff_1d.append(row['1d_sell_sold'] - row['1d_sell_listed'])
    buy_SL_diff_7d.append(row['7d_buy_sold'] - row['7d_buy_listed'])
    buy_SL_diff_1d.append(row['1d_buy_sold'] - row['1d_buy_listed'])

    sell_sold_5pct_1d.append(row['1d_sell_sold'] * 0.05)
    profit_est.append(row['profit'] * row['1d_sell_sold'] * 0.05)

df['sell_SL_diff_7d'] = sell_SL_diff_7d
df['sell_SL_diff_1d'] = sell_SL_diff_1d
df['buy_SL_diff_7d'] = buy_SL_diff_7d
df['buy_SL_diff_1d'] = buy_SL_diff_1d

df['ss_speed'] = sell_sold_5pct_1d
df['profit_est'] = profit_est

df['roi'] = df['roi'] * 100

# df.round()
# df.round({"roi":2})

df = df.round(decimals=0).astype(object)
df = df.convert_dtypes()

# df.dtypes

In [89]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
	display(df[[	
		'name',
		'sell_price',
		'buy_price',
		'profit',
		'profit_est',
		'ss_speed',
		'roi',


		# '7d_buy_listed',
		'7d_buy_sold',
		'buy_SL_diff_7d',

		# '1d_buy_listed',
		'1d_buy_sold',
		'buy_SL_diff_1d',

		# '7d_sell_listed',
		'7d_sell_sold',
		'sell_SL_diff_7d',

		# '1d_sell_listed',
		'1d_sell_sold',
		'sell_SL_diff_1d'

		]]

	.loc	[
		(df['profit'] > 5) &
		(df['roi'] > 1) &
		(df['7d_sell_sold'] > 110) &
		(df['1d_sell_sold'] > 20) &
		# (df['7d_buy_sold'] > 500) &
		(df['sell_SL_diff_7d'] >= 0) &
		(df['sell_SL_diff_1d'] > -5) #&
		]

	.sort_values(by='profit_est', ascending=False)

	# .head(30)

	.style
		.hide_index()
		.bar(subset=[	'buy_SL_diff_7d',
				'buy_SL_diff_1d',
				'sell_SL_diff_7d',
				'sell_SL_diff_1d'
				], align='left', color=['#d65f5f', '#5fba7d'])
		.set_properties(**{'background-color': '#8ba6a0'}, subset=['7d_buy_sold', 'buy_SL_diff_7d'])
		.set_properties(**{'background-color': '#7b838f'}, subset=['1d_buy_sold', 'buy_SL_diff_1d'])
		.set_properties(**{'background-color': '#8ba6a0'}, subset=['7d_sell_sold', 'sell_SL_diff_7d'])
		.set_properties(**{'background-color': '#7b838f'}, subset=['1d_sell_sold', 'sell_SL_diff_1d'])
		.set_properties(**{'background-color': '#b8a73b'}, subset=['profit_est'])
	# .shape[0]
	)

name,sell_price,buy_price,profit,profit_est,ss_speed,roi,7d_buy_sold,buy_SL_diff_7d,1d_buy_sold,buy_SL_diff_1d,7d_sell_sold,sell_SL_diff_7d,1d_sell_sold,sell_SL_diff_1d
Assassin's Pearl Rod,64098,4400,50083,297994,6,1138,31,-323,5,-266,155,38,119,57
Rampager's Pearl Broadsword,25195,15351,6065,107350,18,40,60,-628,10,-79,368,61,354,317
Polysaturating Reverberating Infusion (Red),909070,720000,52710,73794,1,7,42,-102,3,-17,126,8,28,17
Valkyrie Pearl Sabre,20000,9772,7228,34694,5,74,565,-297,78,-136,156,58,96,82
Brandspark Ring,19903,4296,12622,33448,3,294,395,-2609,43,-96,369,95,53,36
Superior Rune of the Citadel,6814,4230,1562,31630,20,37,869,-779,33,-208,4128,165,405,154
Hearty Steel Imbued Inscription,7787,4899,1720,25886,15,35,18,5,0,0,415,414,301,301
Rabid Pearl Broadsword,23998,14013,6385,23624,4,46,242,-259,29,-56,385,39,74,15
Berserker's Draconic Coat,80086,50798,17275,22458,1,34,51,-258,5,-49,194,28,26,17
Berserker's Emblazoned Helm,83900,52109,19206,21127,1,37,34,-162,4,-20,140,18,22,11
